# Fabler av Tor Åge Bringsvær

In [45]:
import dhlab.nbtext as nb
import dhlab.module_update as mu
import pandas as pd
mu.css()

In [67]:
mu.code_toggle()

In [27]:
def categorize(list_of_words):
    proprium = [x for x in list_of_words if x[0].upper() == x[0]]
    other = [x for x in list_of_words if not x in proprium]

    return {'propr':proprium,
           'other': other }

def select_form(w):
    fs = [x.split()[0] for x in nb.word_form(w)]
    if 'verb' in fs:
        res = 'verb'
    elif 'subst' in fs:
        res = 'subst'
    elif len(fs) > 0:
        res = fs[0]
    else:
        res = 'unknown'
    return res

def sort_word_forms(words):
    res = {select_form(w):w for w in words}
    return res

def character_words(Character_name, number_of_words=50):
    res = {w:select_form(w) for w in categorize(list(nb.frame_sort(nb.frame(coll_matrix[Character_name]/tot['tot'], 'association')).head(number_of_words).index))['other']}
    result = {cat:[w for w in res if res[w] == cat] for cat in res.values()}
    return result

Henter alle bøkene - viser de øverste treffene

In [10]:
bring = nb.book_corpus(author = 'Bringsv%tor%', limit = 400)
len(bring)

In [69]:
bring.head(10)

,urn,author,title,year
0,2010111508041,"Bringsværd, Tor Åge",Gobi,1991
1,2011071406096,"Bringsværd, Tor Åge",Kaptein Puma og det farlige romuhyret,2005
2,2016021807523,"Bringsværd, Tor Åge",Jørgen Moes vei nr. 13,2006
3,2012031606058,"Bringsværd, Tor Åge",Heimdalls sønner,1987
4,2015042808047,"Bringsværd, Tor Åge",Karsten og Petra,1994
5,2008091800097,"Bringsværd, Tor Åge",Fellini og grevlingmysteriet,1999
6,2007080304025,"Bringsværd, Tor Åge",Bazar,1970
7,2008020404021,"Bringsværd, Tor Åge",Karsten og Petra kjører brannbil,1994
8,2014052606036,"Bringsværd, Tor Åge",Karsten og Petra går på skøyter,2008
9,2016021608094,"Bringsværd, Tor Åge",Landet hvor alt er annerledes,1980


Summerer alle ordene i alle tekstene i korpuset, som skal brukes til sammenligning med bokhylla ellers

In [12]:
bring_agg = nb.aggregate_urns(nb.pure_urn(bring))

In [14]:
bring_df = nb.frame(bring_agg, 'counts')

In [71]:
nb.frame_sort(bring_df).head(5)

,counts
.,0.073112
",",0.027570
og,0.023162
det,0.016430
er,0.015467


Henter 50000 ord fra bokhylla

In [13]:
totals = nb.frame(nb.totals(50000), 'tot')

In [72]:
nb.frame_sort(totals).head(5)

,tot
.,0.058921
",",0.051453
og,0.025297
i,0.021425
det,0.012728


Normaliser og setter dem sammen for et lite display. Her er det høyest frekvens som gjelder, og interessant nok, er det mye punktum og lite komma i Bringsværs produksjon. Kan tyde på barnebøker.

In [15]:
nb.normalize_corpus_dataframe(bring_df)
nb.normalize_corpus_dataframe(totals)

True

In [50]:
nb.frame_sort(pd.concat([bring_df, totals], axis=1, sort=False)).head(10)

,counts,tot
.,0.073112,0.058921
",",0.027570,0.051453
og,0.023162,0.025297
i,0.014587,0.021425
det,0.016430,0.012728
som,0.012791,0.012343
til,0.010013,0.011847
er,0.015467,0.011580
av,0.009245,0.010948
en,0.012906,0.010783


Gjennomfører en sammenligning ved å dividere Bringsværs data på totalen for bokhylla.

In [16]:
compare = lambda x, y = totals['tot']: nb.frame_sort(nb.frame(x/y))

In [18]:
forskjell = compare(bring_df['counts'])

In [52]:
forskjell.head(10)

,0
Khan,88.724155
Nigel,81.561512
ANDERS,72.605932
Mika,71.875797
Loke,62.996699
Hildur,54.135528
Nøff,53.156186
Lester,49.357471
Lilli,40.031855
Ker,38.654280


Grupperer og sorterer resultatet på morfologiske (syntaktiske?) kategorier

In [38]:
categories = categorize(forskjell[forskjell > 5].dropna().index)

In [37]:
forms = {w:select_form(w) for w in categories['other']}

In [36]:
Grouping = {cat:[word for word in forms if forms[word] == cat] for cat in forms.values()}

In [60]:
navn = "## Egennavn \n"
navn += ', '.join(categories['propr'])

In [61]:
mu.printmd(navn)

## Egennavn 
Khan, Nigel, ANDERS, Mika, Loke, Hildur, Nøff, Lester, Lilli, Ker, Dragen, Klen, Moes, Elvira, Wagner, Felix, Noa, Karsten, Munken, Odin, Zeus, Frigg, Thomassen, Ricardo, Harris, Balder, Saba, Hood, Stener, Brumm, Omar, Nana, Suzanne, Frøy, Harold, Maskin, Sebastian, Gudene, Petra, JOHAN, Robin, Ch, Odins, FAR, Blomst, Low, Bing, Frøya, Djevelen, Rikke, Po, Fleming, Billy, Hund, Venter, Grønn, Slår, Ludvik, Wolfgang, Tyr, Aina, Ennå, Koln, Lillebror, Genova, Salomo, Valhall, Jovisst, Hysj, GYLDENDAL, Lar, Lik, Igor, Løp, Antagelig, Kreta, Illustrert, Holder, Buddha, Legger, Spot, Marguerite, Altfor, Ler, Graham, Troll, Langsomt, Gregorius, Marian, Nicholas, MOR, Lenge, Snakker, Hermod, Månen, Ylva, Cornelia, Kam, ALLE, Fattige, Mu, Brent, Nemlig, Tvertimot, Drømmen, Budapest, Dr, Percy, Vera, Jomfruen, Dager, Egentlig, Katten, Merket, Myten, Kaptein, Benson, Maja, Maar, Kaj, Kiev, Himmelens, Eventyret, Holde, Står, Himalaya, Åge, Alene, Trær, Halen, Holdt, Len, Imorgen, Tor, Føttene, Lot, Alexandria, Shelley, Gatene, Tenker, Hverken, Jammen, Slik, Nok, ..., AUGUST, Setter, Skrevet, Rolig, Slipp, Javisst, Egner, Sier, Esther, Winther, Ve, Og, Jeremias, Døden, Høyt, Rart, Ulven, Sitter, Prinsen, Dyrene, Manhattan, Idag, Likevel, Hører, Hel, Verst, Grå, Natt, Vale, Kjente, Rådhuset, Nixon, Hugo, Salomos, Mørket, Potter, Dypt, Tull, Vokser, Hyde, Antonius, Tarzan, Augusta, Uten, Fer, Fuglen, Dronningen, Jensen, Trenger, Pia, Prøver, Kjenner, Burde, Tennene, Hardt, Gabrielle, Tegnet, Latteren, Veggene, Venn, FanF, Gerda, Satt, Forsiktig, Armand, Englene, Hunder, Musikken, Fienden, Trodde, Prøvde, Bestemor, Bena

In [64]:
pos = ""
for cat in Grouping:
    pos += "## " + cat + "\n"
    pos += ', '.join(Grouping[cat]) + "\n"

In [65]:
mu.printmd(pos)

## verb
ryster, roper, rotte, klovn, omfavner, katter, slikker, brøler, hyler, troll, forkledd, ape, rir, klapper, fakler, torvet, hvisker, tennes, kledt, lener, nøler, kryper, gnir, sverm, klør, rotter, spruter, vinker, blunker, klatrer, vrir, klore, snubler, synger, jubler, avbryter, vakler, sparker, bukker, freser, nikker, aper, velter, lytter, vridde, skotter, vennet, dunker, hugger, drømte, vifter, mumler, røvet, mave, stuper, drømmer, trommer, danser, våkner, vintre, omfavne, hakker, drypper, kysser, kniver, gjemmer, løper, tramper, nøste, stønner, ler, svelger, blåser, virret, fortelles, dirrer, tromme, flokker, skyver, spretter, dempes, gråter, dreper, statuer, snuser, bukter, forvandles, logg, smi, skryter, speider, dytter, piper, spir, tøys, gnager, forvandler, leter, spytter, svømmer, skynder, tusj, blør, flykter, klorte, vinger, sperrer, triller, buret, skjelver, drøm, jager, katt, slikke, krøller, harpe, stryker, kniper, pisker, husker, smiler, nynne, vandrer, sover, vrimler, røve, knuses, drager, boble, motsi, vokter, klamrer, piler, hekser, griser, lukker, snur, sluke, knurret, kvister, nøle, klappe, våger, marsjerer, slikket, raker, sluker, hale, tygger, flokket, løfter, røk, skimter, streifer, puster, narr, fjær, hopper, leker, skremmer, slakter, kneler, skvetter, svinger, rykker, stormer, rynker, krøpet, tilber, streker, trader, rygger, glitrer, bøyer, omfavnet, baller, slukker, knekker, myter, svinet, stanser, viker, rusk, hyle, rusler, feier, spådd, rull, krype, røver, gjentar, gapte, hoster, kolliderer, pels, forsto, orker, skraper, kork, griper, revner, skygger, pinner, nikke, risset, kryp, klipper, brøle, skarve, forvandlet, flokk, gris, ruller, rister, klø, pest, drepes, smeller, etterligne, klatre, brister, heks, kjemper, misforstå, gjemme, stokker, forhekset, sier, tørker, krangler, balanserer, hveste, ryker, sprekker, begir, skriker, jakter, stakket, gidder, strammer, dytt, flyr, tenker, ører, vugge, lengter, slange, smører, skremt, brenner, blander, huk, sleper, fletter, myldret, drømt, hamrer, fløyte, skremte, klump, hilser, flimret, innrømmer, hugget, kaster, pleier, hugg, slår, slanger, prikker, fektet, suget, bretter, frykter, svir, hover, herje, gnage, råtne, lyver, kanter, trenet, snuse, bitt, smidd, løpt, forvandle, rusle
## subst
munken, grimaser, stenene, tigeren, kattene, labyrinten, stener, rottene, dragen, labbene, ødemarken, maven, gudene, tiger, kjempen, elefant, prins, uhyret, skapninger, demoner, elefanten, ulver, elefanter, grisen, ulvene, robot, eventyrene, kjortel, labyrint, arken, labben, kamel, legender, snuten, dukken, klåtrer, hammeren, kameler, måne, skapning, oksen, demon, frosk, hodene, scenen, stenen, grotte, krigere, halen, omveier, barnebøker, eventyr, sten, navlen, monster, korstog, drømmen, grenen, løve, djevler, hunder, bjørner, eventyret, sletten, heltene, novellen, maskinene, morfar, gudinnen, okse, skapningen, løven, halvsirkel, esel, fløyten, speilbilde, hornene, rullen, øksen, tronen, røkelse, hjørner, jungel, murene, løver, fakkel, ropene, tåke, marken, guder, ugle, klode, skuespillerne, stamfar, falk, horn, sverdet, pelsen, tyven, katten, nebbet, musene, verdener, klørne, jungelen, bål, krukken, bensinstasjonen, pus, ulv, dyrene, fjærene, novelle, puslespill, vingene, snute, synene, greven, knute, grålysningen, legenden, trekkspill, helter, hvalen, jegerne, spydet, tegneserier, helten, ørene, trådene, knuter, knokler, fakter, mytene, hundene, grisene, teltene, sverd, pilegrimer, haven, stjernene, ungarsk, hammer, prosesjon, jammen, høvding, torden, lemen, krigerne, kråke, grådighet, krampe, røvere, rustning, kroppene, prinsessen, kjerra, muslimene, felttoget, kjepp, aske, stanken, sommerfugler, tyv, vesener, nesebor, virvar, fugler, jomfruer, tungen, smeden, statue, penselen, misforståelser, knuten, manuskriptet, kropper, reiret, sheriffen, ondskapen, skyene, ørnen, lunge, hekken, sne, pungen, hoder, grotten, ridderen, månen, måfå, gåter, harer, hund, gylden, stammen, nesen, skuespillet, tyver, bulter, sæd, faen, fredløs, dronningen, udødelighet, filler, dyret, djevel, tretoppene, svane, overkropp, regnvær, ørken, nebb, flokken, håndleddet, ødeleggelsen, planeten, hjelm, puten, englene, innbinding, skuespiller, hornet, bykset, bokhandel, kunster, mytologi, dagboken, kjeften, blekksprut, smia, flik, galskap
## adj
stumme, mektigere, merkeligste, eventyrlige, digert, mumlende, dinglende, digre, stripete, morsomste, vrient, hylende, diger, vandrende, avvergende, pipende, kåt, trassig, lodne, barbent, fjollete, usynlig, grådige, forbannede, umulige, skurrende, dovent, vitterlig, sleipe, fillete, lumske, livløs, merkelige, hårete, underjordisk, mektigste, hese, gærne, knudrete, tålmodige, resolutt, vanvittige, jævla, underjordiske, groteske, ulykkelige, skjeggete, sovende, innsmigrende, klokere, syngende, fantastiske, gapende, veldige, bent, magiske, rullende, gåtefulle, larmende, rastløse, uvirkelig, nattlige
## unknown
gudenes, fiction, jeF, seF, flyvende, farver, übevegelige, eF, gyldne, istykker, kaa, hverandre., meF, trance, husveggen, sammen., hij, ingenting., tvertimot, zijn, farve, verden., farvet, übevegelig, hidden, deF, ikveld, übegripelig, mumling, barndoms, luftens, embedsmenn
## prep
midtveis, bakenfor
## det
selveste
## adv
naturligvis
